Proszę pobrać plik medicine.txt, zawierający wyniki analizy nowego leku. W dwóch pierwszych kolumnach znajduje się stężenie dwóch składników w próbce krwi, w trzeciej - informacja o tym, czy lek zadziałał. Dane nie są znormalizowane. Proszę znormalizować dane, podzielić je na zbiór uczący i testujący w proporcjach 80-20 (należy pamiętać o proporcjach klas), zaproponować wielowarstwową sieć neuronową i zbadać jej skuteczność dla różnych ilości warstw i neuronów w tych warstwach. Proszę narysować w jaki sposób sieć dokonała podziału w zbiorze dla kilku sieci (zarówno tych z dobrymi, jak i złymi wynikami) oraz jak wygląda poprawny podział zbioru. Proszę również przedstawić wyniki dla 5-8 różnych struktur sieci, wraz z oceną, która z nich najlepiej poradziła sobie z zadaniem klasyfikacji.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

df = pd.read_csv('medicine.txt', sep=',')
scaler = StandardScaler()
df[['Presence 1', 'Presence 2']] = scaler.fit_transform(df[['Presence 1', 'Presence 2']])
X = df[['Presence 1', 'Presence 2']]
y = df['Was medicine effective?']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)
configs = [(10,), (20,), (10, 10), (20, 20), (10, 10, 10), (20, 20, 20), (10, 10, 10, 10), (20, 20, 20, 20), (20, 20, 20, 20, 20)]
for config in configs:
    mlp = MLPClassifier(hidden_layer_sizes=config, max_iter=1000, random_state=1)
    mlp.fit(X_train, y_train)
    predictions = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Configuration: {config}, Accuracy: {accuracy * 100:.2f}%")
    x_min, x_max = X['Presence 1'].min() - .5, X['Presence 1'].max() + .5
    y_min, y_max = X['Presence 2'].min() - .5, X['Presence 2'].max() + .5
    h = .02
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = mlp.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X['Presence 1'], X['Presence 2'], c=y, cmap=plt.cm.Spectral)
    plt.title(f'Neural Network with configuration {config}')
    plt.show()

**Wnioski:**
Dzięki normalizacji, wszystkie cechy mają tę samą skalę, co ułatwia proces uczenia sieci neuronowej.
Liczba warstw i neuronów w sieci neuronowej ma wpływ na jej skuteczność. W tym przypadku, sieci z większą liczbą warstw i neuronów osiągały wyższą skuteczność. Najlepszą skuteczność osiągnęła sieć z konfiguracją (20, 20, 20, 20).
Sieci z większą liczbą warstw i neuronów były w stanie dokonać bardziej skomplikowanego podziału, co prawdopodobnie przyczyniło się do ich wyższej skuteczności.

Proszę pobrać zbiór ręcznie pisanych cyfr z https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits (można to zrobić funkcją datasets.load_digits() w sklearnie). Proszę sprawdzić skuteczność klasyfikacji na tym zbiorze za pomocą wielowarstwowej sieci neuronowej.

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

digits = datasets.load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=1000, random_state=1)
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

**Wnioski:**
Wynik dokładności bliski 100% wskazuje, że model praktycznie idealnie klasyfikuje cyfry na podstawie danych wejściowych. Model prawidłowo nauczył się wzorców w danych treningowych i skutecznie za ich pomocą klasyfikuje dane testowe.


Proszę sprawdzić, jak zmieni się poprawność klasyfikacji na zbiorze ręcznie pisanych cyfr dla różnych architektur sieci, funkcji aktywacji, ilości epok uczenia i algorytmów uczenia. Proszę zbadać wpływ współczynnika uczenia (learning_rate) podczas używania algorytmu SGD. Dla najciekawszych przykładów proszę wypisać macierze pomyłek.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

digits = datasets.load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)
parameters = [
    {'hidden_layer_sizes': (100,), 'solver': 'adam', 'max_iter': 1000},
    {'hidden_layer_sizes': (50,50), 'solver': 'sgd', 'max_iter': 1000, 'learning_rate_init': 0.01},
    {'hidden_layer_sizes': (30,30,30), 'solver': 'sgd', 'max_iter': 1000, 'learning_rate_init': 0.1},
    {'hidden_layer_sizes': (10,10,10,10), 'solver': 'adam', 'max_iter': 1000, 'learning_rate_init': 0.001},
    {'hidden_layer_sizes': (10,10,10,10), 'solver': 'adam', 'max_iter': 10000, 'learning_rate_init': 0.0001}
]
for params in parameters:
    mlp = MLPClassifier(**params, random_state=42)
    mlp.fit(X_train, y_train)
    predictions = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Parameters: {params}")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("Confusion matrix:")
    print(confusion_matrix(y_test, predictions))
    print("\n")

**Wnioski:**
Dokładność klasyfikacji ręcznie pisanych cyfr zwiększa się wraz ze zwiększaniem liczby warstw i neuronów w sieci neuronowej. Najlepsze wyniki uzyskano dla konfiguracji (100,).
Wpływ współczynnika uczenia (learning_rate) podczas używania algorytmu SGD jest również widoczny. Dla learning_rate = 0.0001 i learning_rate = 0.1, dokładność wynosiła odpowiednio 93.33% i 9.72%.


Proszę pobrać zbiór yeast z UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Yeast). Proszę we własnym zakresie dokonać wstępnej analizy i przygotowania tego zbioru (uwaga, wymagana jest zamiana etykiet tekstowych w ostatniej kolumnie na liczbowe - można je zamienić ręcznie albo przy użyciu takich narzędzi jak https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html, należy jednak pamiętać, że nie musi on ułożyć tych etykiet po kolei). Warto zauważyć, że liczności różnych klas wewnątrz zbioru są _bardzo_ nierówne. Proszę spróbować osiągnąć jak najlepsze wyniki i narysować dla nich macierz pomyłek (dla zbioru uczącego i testującego). Czy trafność na poziomie 0.5 dla takiego zbioru jest dobra? Mogą państwo zbadać też czas wykonywania funkcji fit dla różnych konfiguracji sieci.

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import time

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data"
names = ['Sequence Name', 'mcg', 'gvh', 'alm', 'mit', 'erl', 'pox', 'vac', 'nuc', 'class']
df = pd.read_csv(url, names=names, delim_whitespace=True)
print(df.describe())
le = preprocessing.LabelEncoder()
df['class'] = le.fit_transform(df['class'])
X = df.drop(['Sequence Name', 'class'], axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=3000, random_state=1)
start_time = time.time()
mlp.fit(X_train, y_train)
end_time = time.time()
predictions = mlp.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Training time: {end_time - start_time:.2f}s")
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion matrix:")
print(confusion_matrix(y_test, predictions))
print("Confusion matrix (train set):")
print(confusion_matrix(y_train, mlp.predict(X_train)))

**Wnioski:**
Analizując macierze pomyłek, widzimy, że model ma trudności z niektórymi klasami. Na przykład, dla klasy oznaczonej jako '0', model często mylnie klasyfikuje ją jako '7'.
Nierównomierność liczności różnych klas w zbiorze danych jest prawdopodobnie jednym z czynników wpływających na te trudności. 
Model osiąga lepsze wyniki na zbiorze treningowym niż na zbiorze testowym. Może to wskazywać na overfitting, czyli zbyt mocne dopasowanie modelu do danych treningowych kosztem jego zdolności do generalizacji na nowe dane.